In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from tqdm import tqdm
from datetime import datetime
import pytz
import json
import joblib 
import os
import optuna

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load the dataset after the exploratory data analysis
challenge_set_updated = pd.read_csv("./data/challenge_set_updated_v13.csv")
submission_set = pd.read_csv("./data/submission_set.csv")
submission_set_updated = pd.read_csv("./data/submission_set_updated_v13.csv")



In [3]:
# If necessary change this part to test the model before the training process
df = challenge_set_updated.iloc[:,:]
# df = challenge_set_updated.sample(frac=0.001)

# Separating features and target variable
X = df.drop('tow', axis=1)
y = df['tow']



In [4]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# Handle categorical columns by Label Encoding
label_encoder = LabelEncoder()

# Specify the columns that are categorical
categorical_cols = ['adep', 'ades', 'aircraft_type', 'wtc', 'airline', 'offblock_season', 
                    'flight_duration_category', 'adep_region', 'ades_region', 'flight_direction', 
                    'Manufacturer', 'Model_FAA', 'Physical_Class_Engine', 'FAA_Weight']

# Convert these categorical columns into numerical form using Label Encoding
for col in categorical_cols:
    X[col] = label_encoder.fit_transform(X[col])

# Now, you can proceed with training LightGBM with this processed data


In [5]:
import lightgbm as lgb
from lightgbm import early_stopping, log_evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np
import optuna

# Split the data into training and test sets
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [6]:

from lightgbm import LGBMRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import VotingRegressor



# Initialize models with the best parameters
#lgb_model = LGBMRegressor(
#    n_estimators=,
#    learning_rate=,
#    max_depth=,
#    num_leaves=
#)


cat_model = CatBoostRegressor(
    learning_rate= 0.01, 
    reg_lambda= 69.07051080443502, 
    # 'subsample': 0.9831691718361609, 
    random_strength= 16.348436502754343, 
    depth= 11, 
    # 'colsample_bylevel': 0.9434202103659302, 
    min_data_in_leaf= 2, 
    leaf_estimation_iterations= 7
)

xgb_model = XGBRegressor(
    subsample= 1.0,
    reg_lambda= 0.46415888336127775,
    reg_alpha= 0.166810053720005,
    min_child_weight= 4,
    max_depth= 13,
    learning_rate= 0.01,
    gamma= 0.4444444444444444,
    colsample_bytree= 0.6,
    n_estimators= 7060
)

# Ensemble the models
ensemble_model = VotingRegressor(
    estimators=[
#        ('lgb', lgb_model),
        ('xgb', xgb_model),
        ('cat', cat_model)
    ]
)

# Train the ensemble model on the entire dataset
ensemble_model.fit(X_train_full, y_train_full)



0:	learn: 52825.2687130	total: 240ms	remaining: 3m 59s
1:	learn: 52366.7551392	total: 413ms	remaining: 3m 26s
2:	learn: 51877.6028465	total: 641ms	remaining: 3m 32s
3:	learn: 51389.6291453	total: 1.07s	remaining: 4m 26s
4:	learn: 50923.7844113	total: 1.34s	remaining: 4m 25s
5:	learn: 50458.8763041	total: 1.62s	remaining: 4m 28s
6:	learn: 49986.2852519	total: 1.94s	remaining: 4m 35s
7:	learn: 49547.7913710	total: 2.23s	remaining: 4m 36s
8:	learn: 49110.4476458	total: 2.5s	remaining: 4m 35s
9:	learn: 48654.3716973	total: 2.8s	remaining: 4m 37s
10:	learn: 48201.9297048	total: 3.1s	remaining: 4m 38s
11:	learn: 47749.2582360	total: 3.39s	remaining: 4m 39s
12:	learn: 47299.4138440	total: 3.67s	remaining: 4m 38s
13:	learn: 46896.4643744	total: 3.98s	remaining: 4m 39s
14:	learn: 46474.2070668	total: 4.29s	remaining: 4m 42s
15:	learn: 46044.7686158	total: 4.59s	remaining: 4m 42s
16:	learn: 45633.0139675	total: 4.9s	remaining: 4m 43s
17:	learn: 45209.7518177	total: 5.2s	remaining: 4m 43s
18:	lea

VotingRegressor(estimators=[('xgb',
                             XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=0.6, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          gamma=0.4444444444444444,
                                          grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=0.01, max_bin=None,
                                          max_cat_threshold=None,
                                          max_cat_to_onehot=None,
                                          max_delta_step=None, max_depth=13,
                                          max_leaves=None, min_child_weight=4,
                                          missing=nan,
                                          monotone_constraints=None,
                                          multi_strategy=None,
                                          n_estimators=7060, n_jobs=None,
                                          num_parallel_tree=None,
                                          random_state=None, ...)),
                            ('cat',
                             <catboost.core.CatBoostRegressor object at 0x177585a60>)])

In [7]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd



# Make predictions
y_pred = ensemble_model.predict(X_test)

r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))


print(f"Best Model Performance - R^2 Score: {r2:.4f}, RMSE: {rmse:.4f}")


Best Model Performance - R^2 Score: 0.9969, RMSE: 2933.2674


In [8]:


# Set timezone to São Paulo (UTC-3)
saopaulo_tz = pytz.timezone('America/Sao_Paulo')
timestamp = datetime.now(saopaulo_tz).strftime('%Y%m%d_%H%M%S')




In [16]:
submission_set_updated.T



,0,1,2,3,4,5,6,7,8,9,...,105949,105950,105951,105952,105953,105954,105955,105956,105957,105958
adep,LTFJ,EBBR,KMIA,EGCN,EIDW,LTFJ,EGLL,LSZH,EGLL,LEMD,...,BKPR,LTFJ,KORD,KJFK,KSFO,LTFJ,LOWW,LTFM,EHAM,UBBB
ades,LFLL,KJFK,EGLL,LEAL,LFLL,LTAC,EIDW,EDDH,EICK,EIDW,...,LSGG,LFLL,EGLL,LEBL,EIDW,EKCH,KIAD,LSZH,EDDF,LTFM
aircraft_type,B738,A333,B77W,B38M,A320,B738,A320,A320,A320,A320,...,BCS3,B738,B789,B772,A333,B38M,B763,A321,A320,B738
wtc,M,H,H,M,M,M,M,M,M,M,...,M,M,H,H,H,M,H,M,M,M
airline,6351ec1b849adacc0cbb3b1313d8d39b,bdeeef3a675587d530de70a25d7118d2,5543e4dc327359ffaf5b9c0e6faaf0e1,3922524069809ac4326134429751e26f,a73f82288988b79be490c6322f4c32ed,6351ec1b849adacc0cbb3b1313d8d39b,a73f82288988b79be490c6322f4c32ed,2d5def0a5a844b343ba1b7cc9cb28fa9,a73f82288988b79be490c6322f4c32ed,a73f82288988b79be490c6322f4c32ed,...,2d5def0a5a844b343ba1b7cc9cb28fa9,6351ec1b849adacc0cbb3b1313d8d39b,5543e4dc327359ffaf5b9c0e6faaf0e1,5543e4dc327359ffaf5b9c0e6faaf0e1,a73f82288988b79be490c6322f4c32ed,6351ec1b849adacc0cbb3b1313d8d39b,5d407cb11cc29578cc3e292e743f5393,6351ec1b849adacc0cbb3b1313d8d39b,f502877cab405652cf0dd70c2213e730,6351ec1b849adacc0cbb3b1313d8d39b
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
bearing,293.477205,291.395141,43.036806,178.644825,131.790949,104.173834,301.052709,7.8847,277.346455,352.849559,...,294.105445,293.477205,47.859294,62.377552,33.907846,329.024194,300.682568,300.085737,132.448927,279.85055
elevation_gradient,-0.030154,-0.009004,0.003095,-0.006508,0.149765,1.983765,0.10915,-0.598866,0.232363,-0.368588,...,-1.085058,-0.030154,-0.028218,0.0,0.008554,-0.150159,-0.012142,0.061473,0.311654,0.179499
adep_geo_cluster,11,6,12,13,0,11,13,2,13,7,...,3,11,15,1,4,11,16,11,6,10
ades_geo_cluster,17,1,13,19,17,11,0,9,0,0,...,17,17,13,19,0,9,1,2,2,11


In [18]:
# Use the final model to predict the `tow` for the submission_set_updated
submission_set_features = submission_set_updated.iloc[:,:-1]


In [19]:
# For example, if you used one-hot encoding:
submission_set_features_encoded = pd.get_dummies(submission_set_features)

# Step 2: Align the submission set features with the training set features used in the model
# Find any missing columns in the submission set and add them with default values (0)
missing_cols = set(ensemble_model.feature_names_in_) - set(submission_set_features_encoded.columns)
for col in missing_cols:
    submission_set_features_encoded[col] = 0

# Drop any extra columns that are not in the model's training set
submission_set_features_encoded = submission_set_features_encoded[ensemble_model.feature_names_in_]

# Step 3: Make predictions using the VotingRegressor
submission_set['tow'] = ensemble_model.predict(submission_set_features_encoded)

# Display the resulting DataFrame with predictions
submission_set

,flight_id,date,callsign,adep,name_adep,country_code_adep,ades,name_ades,country_code_ades,actual_offblock_time,arrival_time,aircraft_type,wtc,airline,flight_duration,taxiout_time,flown_distance,tow
0,248753821,2022-01-01,3b3de0f3ad0ee192513995c02f7bf7cf,LTFJ,Istanbul Sabiha Gokcen,TR,LFLL,Lyon,FR,2022-01-01T09:44:00Z,2022-01-01T12:48:33Z,B738,M,6351ec1b849adacc0cbb3b1313d8d39b,170,15,1122,91410.527388
1,248753822,2022-01-01,e06dd03d4a879ca37d9e18c1bd7cad16,EBBR,Brussels,BE,KJFK,New York JFK,US,2022-01-01T09:45:00Z,2022-01-01T17:49:51Z,A333,H,bdeeef3a675587d530de70a25d7118d2,470,15,3205,201489.987737
2,248754498,2022-01-01,2d3b1c962c78c4ebeef11bcd51b9e94c,KMIA,Miami,US,EGLL,London Heathrow,GB,2022-01-01T01:52:00Z,2022-01-01T09:55:16Z,B77W,H,5543e4dc327359ffaf5b9c0e6faaf0e1,473,10,3965,210024.141468
3,248757623,2022-01-01,81564432d3ee97c4bdf4cd8f006753dc,EGCN,Doncaster Sheffield,GB,LEAL,Alicante,ES,2022-01-01T08:20:00Z,2022-01-01T11:06:08Z,B38M,M,3922524069809ac4326134429751e26f,156,10,986,85154.383548
4,248763603,2022-01-01,84be079d7e660db105d91f600b4b3d59,EIDW,Dublin,IE,LFLL,Lyon,FR,2022-01-01T11:01:00Z,2022-01-01T13:00:43Z,A320,M,a73f82288988b79be490c6322f4c32ed,105,15,686,84115.442413
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105954,258066302,2022-12-31,2d3b4446c4d05a25196a9d52cab936fb,LTFJ,Istanbul Sabiha Gokcen,TR,EKCH,Copenhagen,DK,2022-12-31T09:36:00Z,2022-12-31T13:12:17Z,B38M,M,6351ec1b849adacc0cbb3b1313d8d39b,201,15,1199,92493.214735
105955,258068609,2022-12-31,253fd692ed441fac523081471c067772,LOWW,Vienna,AT,KIAD,Washington Dulles,US,2022-12-31T09:49:00Z,2022-12-31T19:38:26Z,B763,H,5d407cb11cc29578cc3e292e743f5393,575,14,3937,176209.280771
105956,258068876,2022-12-31,c9fca302ca2e28acab0eb0bb1b46f11b,LTFM,iGA Istanbul,TR,LSZH,Zurich,CH,2022-12-31T09:25:00Z,2022-12-31T12:24:24Z,A321,M,6351ec1b849adacc0cbb3b1313d8d39b,154,25,988,96730.844893
105957,258064675,2022-12-31,00f96ad0e382476649574ba044c764fc,EHAM,Amsterdam,NL,EDDF,Frankfurt,DE,2022-12-31T10:04:21Z,2022-12-31T10:55:35Z,A320,M,f502877cab405652cf0dd70c2213e730,42,9,240,76844.453233


In [20]:
# Define the submissions directory and create it if it doesn't exist
submissions_dir = 'submissions'
os.makedirs(submissions_dir, exist_ok=True)

# Save the submission with a timestamp in the filename
submission_file = os.path.join(submissions_dir, f"submission_{timestamp}.csv")
submission_set.to_csv(submission_file, index=False)